# <strong> 4a: Gesture Annotation Notebook

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Load the dataset
file_path = '/n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/speed_and_upsample/SN6014_ARTGEST_ENG_VIDAUD/SN6014_ARTGEST_ENG_G_CVC_EVENTS_FOUND.csv'
df = pd.read_csv(file_path)

In [ ]:
from scipy.signal import peak_prominences

# Correcting the previous cell by importing the required function
# and redefining the function to find the two highest peaks and mark intervals
def mark_peak_intervals(df):
    # Create a DataFrame to store peaks and min values
    peaks_df = pd.DataFrame(index=df.index)
    peaks_df['gesture_id'] = df['gesture_id']
    peaks_df['peaks'] = 0
    peaks_df['min'] = 0

    # Group by 'gesture_id' and find peaks for each gesture
    for gesture_id, group in df.groupby('gesture_id'):
        if group.shape[0] > 3:  # Need at least 3 points to define two peaks
            # Assuming 'right_wrist_speed' is the column to analyze for peaks
            speeds = group['right_wrist_speed'].values
            peaks, _ = find_peaks(speeds, distance=1)
            # Sort peaks by prominence and take the two highest
            prominences = peak_prominences(speeds, peaks)[0]
            sorted_peaks = peaks[np.argsort(prominences)[-2:]]
            # Mark the intervals between the two highest peaks
            if sorted_peaks.size == 2:
                peaks_df.loc[group.index[min(sorted_peaks):max(sorted_peaks)+1], 'peaks'] = 1
                # Find the absolute smallest valley within the peak interval
                valley = np.argmin(speeds[min(sorted_peaks):max(sorted_peaks)+1]) + min(sorted_peaks)
                peaks_df.loc[group.index[valley], 'min'] = 1

    return peaks_df

# Apply the function to the df
peaks_data = mark_peak_intervals(df)
peaks_data.head()

In [ ]:
# Align the peaks df with the original df
df['peaks'] = peaks_data['peaks']
df['min'] = peaks_data['min']
df.head()

In [ ]:
# # plot the results using plotly.graph_objects
# import plotly.graph_objects as go

# # Filter the df to only include rows where 'min' and 'peaks' are equal to 1
# min_df = df[df['min'] == 1]
# peaks_df = df[df['peaks'] == 1]

# fig = go.Figure()
# fig.add_trace(go.Scatter(x=df['time_ms'], y=df['right_wrist_speed'], name='Speed'))
# fig.add_trace(go.Scatter(x=peaks_df['time_ms'], y=peaks_df['right_wrist_speed'], name='Peaks', mode='markers'))
# fig.add_trace(go.Scatter(x=min_df['time_ms'], y=min_df['right_wrist_speed'], name='Min', mode='markers'))
# fig.update_layout(title='Speed with Peaks and Min Values',
#                     # change size of the figure
#                     width=1000,
#                     height=800)
# fig.show()


In [ ]:
def mark_crossings(df):
    # Add columns for crossing points
    df['right_wrist_delta_x_cross'] = 0
    df['right_wrist_delta_y_cross'] = 0

    # Group by 'gesture_id' and find crossings for each gesture
    for gesture_id, group in df.groupby('gesture_id'):
        if gesture_id != 'nan':  # Skip gesture_id = 0
            # Find where the right wrist delta x crosses 0
            delta_x = group['right_wrist_delta_x'].values
            cross_x = np.where(np.diff(np.sign(delta_x)))[0]
            df.loc[group.index[cross_x], 'right_wrist_delta_x_cross'] = 1

            # Find where the right wrist delta y crosses 0
            delta_y = group['right_wrist_delta_y'].values
            cross_y = np.where(np.diff(np.sign(delta_y)))[0]
            df.loc[group.index[cross_y], 'right_wrist_delta_y_cross'] = 1

    return df

# Apply the function to the data and peaks_df
crossings_data = mark_crossings(df)
crossings_data.head()


In [ ]:
# Align the crossings df with the original df
df['right_wrist_delta_x_cross'] = crossings_data['right_wrist_delta_x_cross']
df['right_wrist_delta_y_cross'] = crossings_data['right_wrist_delta_y_cross']


In [ ]:
# # plot the results using plotly.graph_objects
# import plotly.graph_objects as go

# # Filter the df to only include rows where 'min' and 'peaks' are equal to 1
# cross_x_df = df[df['right_wrist_delta_x_cross'] == 1]
# cross_y_df = df[df['right_wrist_delta_y_cross'] == 1]

# fig = go.Figure()
# fig.add_trace(go.Scatter(x=df['time_ms'], y=df['right_wrist_delta_x'], name='Delta X'))
# fig.add_trace(go.Scatter(x=df['time_ms'], y=df['right_wrist_delta_y'], name='Delta Y'))

# # Add trace for speed data
# # fig.add_trace(go.Scatter(x=df['time_ms'], y=df['right_wrist_speed'], name='Speed', yaxis='y2'))

# fig.add_trace(go.Scatter(x=cross_x_df['time_ms'], y=cross_x_df['right_wrist_delta_x'], name='Cross X', mode='markers'))
# fig.add_trace(go.Scatter(x=cross_y_df['time_ms'], y=cross_y_df['right_wrist_delta_y'], name='Cross Y', mode='markers'))
# # Add the min values to the plot as well but only along the x axis not on the speed trace
# fig.add_trace(go.Scatter(x=min_df['time_ms'], y=min_df['right_wrist_delta_x'], name='Min', mode='markers'))


# # Mark each gesture event with a box
# for gesture_id, group in df.groupby('gesture_id'):
#     if gesture_id != 'nan':  # Skip gesture_id = 0
#         # Find the start and end times for each gesture
#         start_time = group['time_ms'].min()
#         end_time = group['time_ms'].max()
#         # Add a box to the plot for each gesture
#         fig.add_shape(type='rect',
#                     xref='x',
#                     yref='paper',
#                     x0=start_time,
#                     y0=0,
#                     x1=end_time,
#                     y1=1,
#                     line=dict(color='Red', width=2),
#                     fillcolor='LightSkyBlue',
#                     opacity=0.5,
#                     layer='below')

# # Mark the intervals between the peaks with a different color box
# for gesture_id, group in df.groupby('gesture_id'):
#     if gesture_id != 'nan':  # Skip gesture_id = 0
#         # Find the start and end times for each gesture
#         start_time = group[group['peaks'] == 1]['time_ms'].min()
#         end_time = group[group['peaks'] == 1]['time_ms'].max()
#         # Add a box to the plot for each gesture
#         fig.add_shape(type='rect',
#                     xref='x',
#                     yref='paper',
#                     x0=start_time,
#                     y0=0,
#                     x1=end_time,
#                     y1=1,
#                     line=dict(color='Red', width=2),
#                     fillcolor='LightGreen',
#                     opacity=0.5,
#                     layer='below')

# fig.update_layout(title='Speed with Crossings',
#                 # Make yaxis1 range based on the deltas
#                 yaxis=dict(range=[min(df['right_wrist_delta_x'].min(), df['right_wrist_delta_y'].min()),
#                                 max(df['right_wrist_delta_x'].max(), df['right_wrist_delta_y'].max())]),
#                 width=1400,
#                 height=600)
# fig.show()

In [ ]:
df.to_csv('KEYPOINTS/SN6014_ARTGEST_ENG_G_CVC_FINAL.csv')